In [1]:
from pyquil.quil import *
from pyquil.api import get_qc
from pyquil.gates import *
from pyquil.latex import display, to_latex
from pyquil.simulation.tools import lifted_gate, program_unitary, lifted_gate_matrix

In [1]:
from functions import *

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
qc = get_qc("9q-square-qvm")

# Programmable recipe
![alt text](rotation_definition.png "Title")
$$R(\theta, \phi) = R_z(-\frac{\phi}{2})R_y(\theta)R_z(\frac{\phi}{2})$$
![alt text](programmble_decomposition.png "Title")

In [5]:
# program_unitary(G, n_qubits=2)

$$R(\theta, \phi) = R_z(-\frac{\phi}{2})R_y(\theta)R_z(\frac{\phi}{2})\\
U(2) = R(\theta, \phi) R_z(\phi_z)\\
or\\
U(2) = R_z(\theta) R_y(\phi) R_z(\psi)
$$

In [3]:
G = Program( CPHASE01(-np.pi/2, control=0, target=1), CPHASE10(-np.pi/2, control=0, target=1) )

In [4]:
program_unitary(Program(H(1), G), n_qubits=2)

array([[ 7.07106781e-01+0.j        ,  0.00000000e+00+0.j        ,
         7.07106781e-01+0.j        ,  0.00000000e+00+0.j        ],
       [ 0.00000000e+00+0.j        ,  4.32978028e-17-0.70710678j,
         0.00000000e+00+0.j        ,  4.32978028e-17-0.70710678j],
       [ 4.32978028e-17-0.70710678j,  0.00000000e+00+0.j        ,
        -4.32978028e-17+0.70710678j,  0.00000000e+00+0.j        ],
       [ 0.00000000e+00+0.j        ,  7.07106781e-01+0.j        ,
         0.00000000e+00+0.j        , -7.07106781e-01+0.j        ]])

In [8]:
Program(G)

In [5]:
# %%writefile -a functions.py
G = Program( CPHASE01(-np.pi/2, control=0, target=1), CPHASE10(-np.pi/2, control=0, target=1) )

def arbitary_single_qubit_circuit(theta, phi, si, qubit):
    return Program( RZ(si, qubit = qubit), RY(phi, qubit = qubit), RZ(theta, qubit = qubit) )

def r_theta_phi_rotation(theta, phi, qubit):
    return arbitary_single_qubit_circuit( - phi/2, theta, phi/2, qubit)

def give_random_single_qubit_gate(qubit):
    theta, si = np.random.uniform(0,2*np.pi, size = 2)
    
    phi_range = np.linspace(0,np.pi)
    p_phi = np.sin(phi_range) / np.sum(np.sin(phi_range))
    phi = np.random.choice(phi_range, p = p_phi)
    return arbitary_single_qubit_circuit(theta, phi, si, qubit = qubit)

def normalized_abs_angle_dist(angle_range):
    dist = np.pi - np.abs( np.pi - angle_range )
    dist /= np.sum(dist)
    return dist

# def give_v_circuit(alpha, beta, delta, qubits = [0,1]):
#     prog = Program(G,  r_theta_phi_rotation(alpha, 0, qubit =qubits[0]),
#                    r_theta_phi_rotation(3*np.pi/2,0, qubit =qubits[1]), G)
#     prog += Program( r_theta_phi_rotation(beta, np.pi/2, qubit = qubits[0]), 
#                     r_theta_phi_rotation(3*np.pi/2, delta, qubit = qubits[1]), G)
#     return prog

# def give_v_circuit(alpha, beta, delta, qubits = [0,1]):
#     prog = Program(CNOT(control=qubits[1], target=qubits[0]),
#                    r_theta_phi_rotation(alpha, 0, qubit =qubits[0]),
#                    r_theta_phi_rotation(3*np.pi/2,0, qubit =qubits[1]),
#                    CNOT(control=qubits[0], target=qubits[1]))
#     prog += Program( r_theta_phi_rotation(beta, np.pi/2, qubit = qubits[0]),
#                     r_theta_phi_rotation(3*np.pi/2, delta, qubit = qubits[1]),
#                     CNOT(control=qubits[1], target=qubits[0]))
#     return prog

def give_v_circuit(alpha, beta, delta, qubits = [0,1]):
    prog = Program(CNOT(control=qubits[1], target=qubits[0]),
                   RZ(angle = delta, qubit =qubits[0]),
                   RY(beta, qubit =qubits[1]),
                   CNOT(control=qubits[0], target=qubits[1]))
    prog += Program(RY(angle= alpha, qubit = qubits[1]),
                    CNOT(control=qubits[1], target=qubits[0]))
    return prog

def give_random_two_quibt_circuit(qubits):
    a,b,c,d = [give_random_single_qubit_gate(qubit=qubit) for _ in range(2) for qubit in qubits]
    
    angles_range = np.linspace(0,2*np.pi)
    alpha, beta, delta = np.random.choice(angles_range, p = normalized_abs_angle_dist(angles_range),
                                          size = 3)
    
    prog = Program(a, b )
    prog += give_v_circuit(alpha, beta, delta, qubits = [0,1])
    prog += Program(c, d )
    return prog

In [6]:
program_unitary(give_random_two_quibt_circuit([0,1]), n_qubits=2)

array([[-0.52836531-2.98592791e-04j,  0.49947079+3.33714326e-01j,
        -0.01522696-4.31570689e-01j, -0.38726747+1.53402989e-01j],
       [-0.25236647+5.42764074e-01j, -0.60049094-3.51216911e-02j,
        -0.04861536-4.42640448e-01j, -0.0748518 -2.75678456e-01j],
       [ 0.25530508-4.95049667e-01j, -0.02552895-4.07138669e-01j,
        -0.43912659-4.72198754e-01j, -0.26482265-1.93382158e-01j],
       [-0.21782815-6.96015702e-02j,  0.20862568+2.59619871e-01j,
        -0.44636352-1.32907543e-02j,  0.62627007-4.95124361e-01j]])

In [7]:
sample = give_random_two_quibt_circuit([0,1])
# print( qc.compile( sample ) , len(sample), len(qc.compile( sample )))

In [7]:
program_unitary(G, n_qubits=2)

array([[1.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 6.123234e-17-1.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 6.123234e-17-1.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        1.000000e+00+0.j]])

# Visualization
## single-qubit

In [ ]:
num_samples = 2021

single_qubit_unitary_samples = [program_unitary(give_random_single_qubit_gate(qubit=0), n_qubits = 1) for _ in range(num_samples)]

In [ ]:
single_zero_state_density_matrix = np.array([[1,0],[0,0]]) #|0><0| state
# What is U|0><0|U^T
single_final_states = np.array( [np.dot( np.dot(u,single_zero_state_density_matrix), u.conj().T ) for u in single_qubit_unitary_samples] )

In [ ]:
# lets see where U|0><0|U^T is likely found
single_qubit_bloch_vectors = np.array([convert_to_bloch_vector(s) for s in single_final_states])

In [ ]:
plot_bloch_sphere(single_qubit_bloch_vectors)

## Two qubits

In [ ]:
num_samples = 2021

two_qubit_unitary_samples = [program_unitary(give_random_two_quibt_circuit([0,1]), n_qubits = 2) for _ in range(num_samples)]

In [ ]:
two_zero_state_density_matrix = np.kron( np.array([[1,0],[0,0]]) , np.array([[1,0],[0,0]]) )
two_final_states = np.array( [np.dot( np.dot(u,two_zero_state_density_matrix), u.conj().T ) for u in two_qubit_unitary_samples] )

In [ ]:
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
I = np.eye(2)

two_qubit_unitary_basis = np.array([ [ np.kron(x,y) for x in [X,Y,Z] ] for y in [X,Y,Z] ])
# two_qubit_unitary_basis[0]

In [ ]:
# Used the mixed state simulator so we could have the density matrix for this part!
def reduce_to_bloch_vector(rho, sigma_arr:np.array):
    """Reduce a density matrix to a Bloch vector."""
    ax = np.trace(np.dot(rho, sigma_arr[0])).real
    ay = np.trace(np.dot(rho, sigma_arr[1])).real
    az = np.trace(np.dot(rho, sigma_arr[2])).real
    return [ax, ay, az]

In [ ]:
two_qubit_bloch_vectors = np.array([reduce_to_bloch_vector(s,two_qubit_unitary_basis[2]) for s in two_final_states])
plot_bloch_sphere(two_qubit_bloch_vectors)

### Verification of Haar distribution
we need to find out whether the distribution of the points is really uniform over the total space. To do so we first compute the radial distance of points and plot their cumulative histogram. If it grows with the power of 3 then we can be sure that the density of points is uniform in the sphere.

In [ ]:
r_bloch_vectors = np.sqrt( two_qubit_bloch_vectors[:,0]**2 + two_qubit_bloch_vectors[:,1]**2 + two_qubit_bloch_vectors[:,2]**2)

In [ ]:
n_bins = 10

fig, ax = plt.subplots()

bins = np.logspace(np.log10(0.1),np.log10(1),n_bins)
# plot the cumulative histogram
pop, bins, hist_ = ax.hist( r_bloch_vectors, bins, histtype='step',
                           cumulative=True, density = True, label='Empirical')
ax.set_yscale('log')
ax.set_xscale('log')
# ax.plot(np.linspace(0,1,10), np.linspace(0,1,10)**slope)
ax.plot(bins, bins**3)
plt.show()


In [ ]:
non_zero_mask = pop != 0
regression_pop = pop[non_zero_mask]
regression_bins = np.array( bins[:-1] ) [non_zero_mask]
exp, intercept = np.polyfit(np.log(regression_bins), np.log(regression_pop), deg = 1)
exp, intercept

In [ ]:
def exponent_of_dist(data):
    n_bins = 100
    bins = np.logspace(np.log10(0.1),np.log10(1),n_bins)
    # plot the cumulative histogram
    pop, bins, hist_ = plt.hist( data, bins, cumulative=True, density = True)
    # extrapolate the exponent
    non_zero_mask = pop != 0
    regression_pop = pop[non_zero_mask]
    regression_bins = np.array( bins[:-1] ) [non_zero_mask]
    exp, intercept = np.polyfit(np.log(regression_bins), np.log(regression_pop), deg = 1)
    plt.close()
    return exp

In [ ]:
exp_list = []
for i in range(len(two_qubit_unitary_basis)):
    two_qubit_bloch_vectors = np.array([reduce_to_bloch_vector(s,two_qubit_unitary_basis[i]) for s in two_final_states])
    r_bloch_vectors = np.sqrt( two_qubit_bloch_vectors[:,0]**2 + two_qubit_bloch_vectors[:,1]**2 + two_qubit_bloch_vectors[:,2]**2)
    exp_list.append(exponent_of_dist(r_bloch_vectors))
exp_list

# Matrix decomposition verification


In [30]:
from scipy.stats import unitary_group
import cmath

In [31]:
u_matrix = unitary_group.rvs(4)
lambda_unitary = np.array([ [1, 1j , 0 , 0],[0, 0, 1j, 1],[0, 0, 1j, -1],[1, -1j, 0, 0] ]) / np.sqrt(2)

In [32]:
def matrix_in_magic_basis(matrix):
    return np.dot( lambda_unitary.conj().transpose(), np.dot(matrix, lambda_unitary) )

def matrix_out_magic_basis(magic_matrix):
    return np.dot( lambda_unitary, np.dot(magic_matrix, lambda_unitary.conj().transpose()) )

def relative_phases(complex_arr:np.array):
    phases = np.array( [cmath.phase(x) for x in complex_arr] )
    phases = np.sort(phases)
    phases -= phases[0]
    return phases

def strip_global_factor(matrix):
    shape_length = np.shape(matrix)[0]
    return matrix / abs(np.linalg.det(matrix))**(1/shape_length)

def strip_global_phase(matrix):
    shape_length = np.shape(matrix)[0]
    return matrix / np.linalg.det(matrix)**(1/shape_length)

def get_ordered_eig(matrix):
    values, vecs = np.linalg.eig(matrix)
    order = np.argsort([cmath.phase(x) for x in values])
    values = values[order]
    vecs = np.transpose(vecs.transpose()[order])
    return values, vecs

Strip U of any global phase

In [33]:
u_matrix /= ( np.linalg.det(u_matrix) )**(1/4)

In [34]:
u_magic_matrix = matrix_in_magic_basis(u_matrix)

In [35]:
u_u_T = np.dot(u_magic_matrix, u_magic_matrix.transpose())

In [36]:
u_u_T_eigen_values, u_u_T_eigen_vectors = get_ordered_eig(u_u_T)
# u_u_T_eigen_values, u_u_T_eigen_vectors = np.linalg.eig(u_u_T)

#### V Gate:

In [37]:
# eigen_values_phases = relative_phases(u_u_T_eigen_values)[1:]
eigen_values_phases = [cmath.phase(x) for x in u_u_T_eigen_values]
alpha, beta, delta = np.array([eigen_values_phases[0] + eigen_values_phases[1],
                               eigen_values_phases[0] + eigen_values_phases[2],
                               eigen_values_phases[1] + eigen_values_phases[2] ]) / 2
v_matrix = program_unitary(give_v_circuit(alpha, beta, delta), n_qubits=2)
v_matrix

array([[ 0.39198462-0.05044008j,  0.        +0.j        ,
         0.        +0.j        ,  0.91107604-0.11723611j],
       [ 0.        +0.j        ,  0.38858189+0.05000222j,
         0.91253253+0.11742353j,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.91253253+0.11742353j,
        -0.38858189-0.05000222j,  0.        +0.j        ],
       [-0.91107604+0.11723611j,  0.        +0.j        ,
         0.        +0.j        ,  0.39198462-0.05044008j]])

In [38]:
v_magic_matrix = matrix_in_magic_basis(v_matrix)
# v_magic_matrix = matrix_in_magic_basis(np.e**(-1j * np.pi/4) * v_matrix)
v_v_T = np.dot(v_magic_matrix, v_magic_matrix.transpose())

Check whether vv^T and uu^T have same eigenvalues

In [39]:
v_v_T_eigen_values, v_v_T_eigen_vectors = get_ordered_eig(v_v_T)

In [40]:
relative_phases(v_v_T_eigen_values), relative_phases(u_u_T_eigen_values)

(array([0.        , 2.03574382, 3.64603082, 4.65797085]),
 array([0.        , 2.03574382, 3.64603082, 4.65797085]))

In [41]:
u_u_T_eigen_values, v_v_T_eigen_values

(array([-0.84902651-0.52835025j,  0.85294629-0.52199869j,
         0.48791704+0.87289j   , -0.4813885 +0.87650734j]),
 array([-0.84902651-0.52835025j,  0.85294629-0.52199869j,
         0.48791704+0.87289j   , -0.4813885 +0.87650734j]))

#### K and L

In [43]:
v_v_T_eigen_values, v_v_T_eigen_vectors = get_ordered_eig(v_v_T)

In [44]:
k_matrix = np.copy(v_v_T_eigen_vectors.transpose()) # transpose needed to be consistent with the paper
l_matrix = np.copy(u_u_T_eigen_vectors.transpose())

In [46]:
np.matmul(k_matrix, np.matmul(v_v_T, k_matrix.transpose()))

array([[-8.49026508e-01-5.28350252e-01j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, -1.06286914e-16+5.15558087e-17j],
       [ 0.00000000e+00+0.00000000e+00j,  8.52946286e-01-5.21998691e-01j,
         9.11763242e-17+9.81748372e-17j,  0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j,  1.10493961e-16+6.93082939e-17j,
         4.87917045e-01+8.72890003e-01j,  0.00000000e+00+0.00000000e+00j],
       [-1.21359748e-16+1.48961413e-17j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, -4.81388501e-01+8.76507336e-01j]])

In [47]:
np.matmul(l_matrix, np.matmul(u_u_T, l_matrix.transpose()))

array([[-8.49026508e-01-5.28350252e-01j, -1.45363065e-16+2.64501221e-16j,
         4.03941742e-18+1.17789652e-16j, -3.99851793e-16+6.15448739e-17j],
       [-9.24388155e-17+2.48201334e-16j,  8.52946286e-01-5.21998691e-01j,
         5.24540647e-17-1.11076743e-16j,  1.27018074e-16-2.48965340e-16j],
       [ 7.50562494e-17+3.21357216e-17j,  1.10250307e-16-3.85331257e-17j,
         4.87917045e-01+8.72890003e-01j, -1.35698975e-16+2.65646028e-16j],
       [-3.72457982e-16+5.65691863e-18j,  1.35688700e-16-2.50062034e-16j,
        -1.45648966e-16+2.46822875e-16j, -4.81388501e-01+8.76507336e-01j]])

#### AB
$$ A \otimes B = \Lambda ( v^\dagger k^T l u ) \Lambda^\dagger $$

In [48]:
a_tensor_b = matrix_out_magic_basis( np.matmul( v_magic_matrix.conjugate().transpose(),
                                               np.matmul(k_matrix.transpose(), np.matmul(l_matrix, u_magic_matrix))) )
a_tensor_b

array([[-0.31537042-0.19032003j, -0.3470467 -0.29366903j,
         0.32997065+0.38954351j,  0.32583287+0.53930051j],
       [-0.3395175 -0.38125142j, -0.33909012-0.53106496j,
        -0.14429941-0.33890692j, -0.10948869-0.44124272j],
       [ 0.10948869-0.44124272j, -0.14429941+0.33890692j,
        -0.33909012+0.53106496j,  0.3395175 -0.38125142j],
       [ 0.32583287-0.53930051j, -0.32997065+0.38954351j,
         0.3470467 -0.29366903j, -0.31537042+0.19032003j]])

### Solving for single parts

In [52]:
zero_state = np.array([[1],[0]])
one_state = np.array([[0],[1]])
def partial_trace_on_left(one_tensor_two):
    states_id_mat = [np.kron(state, np.eye(2)) for state in [zero_state, one_state]]
    tr_one_tensor_two =  np.zeros((2,2))
    for vec_id in states_id_mat:
        tr_one_tensor_two = tr_one_tensor_two +  np.matmul(vec_id.conj().transpose(), np.matmul(one_tensor_two, vec_id))
    return tr_one_tensor_two

def partial_trace_on_right(one_tensor_two):
    states_id_mat = [np.kron(np.eye(2), state) for state in [zero_state, one_state]]
    tr_one_tensor_two =  np.zeros((2,2))
    for vec_id in states_id_mat:
        tr_one_tensor_two = tr_one_tensor_two + np.matmul(vec_id.conj().transpose(), np.matmul(one_tensor_two, vec_id))
    return tr_one_tensor_two

In [53]:
a = strip_global_phase( strip_global_factor(partial_trace_on_right(a_tensor_b)) )
b = strip_global_phase( strip_global_factor(partial_trace_on_left(a_tensor_b)) )

In [54]:
abs(np.linalg.det(a)), abs(np.linalg.det(b))

(1.0, 1.0)

In [55]:
np.matmul(a, a.conj().transpose())

array([[ 1.00000000e+00-1.56903945e-17j, -2.77555756e-17+0.00000000e+00j],
       [-2.77555756e-17-2.77555756e-17j,  1.00000000e+00+1.80160274e-17j]])

In [56]:
np.matmul(b, b.conj().transpose())

array([[ 1.00000000e+00+7.69882651e-18j, -5.27355937e-16+1.66533454e-16j],
       [-5.27355937e-16-1.66533454e-16j,  1.00000000e+00+4.33650073e-18j]])

#### CD
$$ C \otimes D = \Lambda ( l^T k ) \Lambda^\dagger $$

In [59]:
c_tensor_d = matrix_out_magic_basis( np.matmul(l_matrix.transpose(), k_matrix) )
c_tensor_d

array([[ 0.64108057-0.44681471j, -0.23326932-0.29099978j,
         0.4434575 +0.08478603j,  0.02619822-0.21388616j],
       [ 0.29693838-0.22566136j,  0.51342466+0.58908642j,
         0.21313665+0.03172393j, -0.03640685+0.45001974j],
       [ 0.03640685+0.45001974j,  0.21313665-0.03172393j,
         0.51342466-0.58908642j, -0.29693838-0.22566136j],
       [ 0.02619822+0.21388616j, -0.4434575 +0.08478603j,
         0.23326932-0.29099978j,  0.64108057+0.44681471j]])

In [62]:
c = strip_global_phase( strip_global_factor(partial_trace_on_right(c_tensor_d)))
d = strip_global_phase( strip_global_factor(partial_trace_on_left(c_tensor_d)))
c

array([[ 0.85936494+0.10590105j,  0.30299131+0.39808683j],
       [-0.30299131+0.39808683j,  0.85936494-0.10590105j]])

In [63]:
abs(np.linalg.det(c)), abs(np.linalg.det(d))

(1.0, 1.0000000000000002)

In [65]:
np.matmul(d, d.conj().transpose()), np.matmul(c, c.conj().transpose())

(array([[ 1.0000000e+00+1.50516946e-19j, -4.4408921e-16-3.46944695e-17j],
        [-4.4408921e-16+4.51028104e-17j,  1.0000000e+00+1.09647625e-17j]]),
 array([[1.00000000e+00+1.53441836e-18j, 8.32667268e-17+3.88578059e-16j],
        [8.32667268e-17-3.88578059e-16j, 1.00000000e+00+9.90172102e-18j]]))

#### Construct U
$$ U = (A \otimes B) V (C \otimes D) $$

In [66]:
u_constructed = np.matmul(c_tensor_d, np.matmul(v_matrix, a_tensor_b))
u_constructed

array([[-0.50519748-0.52026263j,  0.02560099+0.09181684j,
         0.35535762-0.44126975j, -0.26269699+0.2738774j ],
       [ 0.20689396-0.32916909j, -0.43999678+0.00206612j,
        -0.06142238-0.4260829j ,  0.66304511-0.17404886j],
       [-0.18327498-0.08081969j, -0.65850948-0.55101128j,
        -0.28879047+0.14601009j, -0.34135959+0.03721622j],
       [ 0.32061157+0.4243013j , -0.20910636+0.12782485j,
         0.12611434-0.61078549j, -0.46771457-0.22224156j]])

In [67]:
u_constructed /= ( np.linalg.det(u_constructed) )**(1/4)

In [68]:
u_constructed

array([[-0.50519748-0.52026263j,  0.02560099+0.09181684j,
         0.35535762-0.44126975j, -0.26269699+0.2738774j ],
       [ 0.20689396-0.32916909j, -0.43999678+0.00206612j,
        -0.06142238-0.4260829j ,  0.66304511-0.17404886j],
       [-0.18327498-0.08081969j, -0.65850948-0.55101128j,
        -0.28879047+0.14601009j, -0.34135959+0.03721622j],
       [ 0.32061157+0.4243013j , -0.20910636+0.12782485j,
         0.12611434-0.61078549j, -0.46771457-0.22224156j]])

In [69]:
u_matrix /= ( np.linalg.det(u_matrix) )**(1/4)
u_matrix

array([[-0.50519748-0.52026263j,  0.02560099+0.09181684j,
         0.35535762-0.44126975j, -0.26269699+0.2738774j ],
       [ 0.20689396-0.32916909j, -0.43999678+0.00206612j,
        -0.06142238-0.4260829j ,  0.66304511-0.17404886j],
       [-0.18327498-0.08081969j, -0.65850948-0.55101128j,
        -0.28879047+0.14601009j, -0.34135959+0.03721622j],
       [ 0.32061157+0.4243013j , -0.20910636+0.12782485j,
         0.12611434-0.61078549j, -0.46771457-0.22224156j]])

#### Check Check
Given U and constructed U are allowed to be different by a phase factor. So we should have their dot to be an identity with a global phase shift.
$$ U_g U^{\dagger}_c  = e^{i\phi I}$$

In [70]:
np.dot(u_matrix, u_constructed.conjugate().transpose())

array([[ 1.00000000e+00-7.63278329e-17j,  1.66533454e-16+2.15105711e-16j,
         8.32667268e-17-3.74700271e-16j,  7.78402950e-16-4.44089210e-16j],
       [ 1.66533454e-16-6.10622664e-16j,  1.00000000e+00+2.77555756e-17j,
         4.71844785e-16-5.82867088e-16j, -2.77555756e-16+5.13478149e-16j],
       [-8.32667268e-17+1.66533454e-16j,  2.35922393e-16+6.93889390e-16j,
         1.00000000e+00+2.22044605e-16j, -3.05311332e-16-3.05311332e-16j],
       [ 2.67364256e-16-2.22044605e-16j, -5.55111512e-17-3.33066907e-16j,
        -3.88578059e-16+7.21644966e-16j,  1.00000000e+00-1.94289029e-16j]])

## Solving for single rotation angles

In [100]:
from scipy.optimize import fsolve, root

In [101]:
def multiply_matrices(*args):
    result = np.eye(2)
    for mat in args:
        result = np.matmul(mat, result)
    return result

def rz(theta):
    return np.array([[np.e**(1j*theta /2), 0 ],[0, np.e**(-1j*theta /2)]])
def ry(theta):
    return np.array([[np.cos(theta/2), np.sin(theta/2)],[- np.sin(theta/2), np.cos(theta/2)]])
def rx(theta):
    return np.array([[np.cos(theta/2), 1j*np.sin(theta/2)],[1j*np.sin(theta/2), np.cos(theta/2)]])

def single_unitary(beta, gamma, delta):
    return multiply_matrices(rz(beta), ry(gamma), rz(delta))

In [108]:
def find_rot_angles(unitary):
    def func(x):
        return  single_unitary(x[0],x[1],x[2]).flatten() -  unitary.flatten()
    result = root(func, [0,0,0], method='lm')
    return result


In [109]:
find_rot_angles(a)

TypeError: Cannot cast array data from dtype('complex128') to dtype('float64') according to the rule 'safe'

error: Result from function call is not a proper array of floats.

In [90]:
fsolve?

In [85]:
single_unitary(0,0,0)[0]

array([1.+0.j, 0.+0.j])